# <font color='lightseagreen'> *Case Study: How Does a Bike-Share Navigate Speedy Success*
---

  > ###  <font color='salmon'>   • Business task statement: Cyclistic is a bike-share company in Chicago. It has two broad classifications of customers, casual riders and annual members. Cyclistic’s finance analysts have concluded that annual members are much more profitable than casual riders. The director of marketing believes the company’s future success depends on maximizing the number of annual memberships. To do that, however, the marketing analyst team, which I am a part of, needs to better understand how annual members and casual riders differ, why casual riders would buy a membership, and how digital media could affect their marketing tactics. As a junior data analyst, my role is to answer the question: How do annual members and casual riders use Cyclistic bikes differently? From the insights we get, we will design an innovative marketing strategy to convert casual riders into annual members.</font>
   
   ---


  > ### <font color='salmon'> • This notebook analyzes a 12-month data to come up with insights that will help provide solution to the business problem above.</font>
    
  > ### <font color='scarlet'> • The project was originally completed in Apr, 2023</font>
  > ### <font color='scarlet'> • Last updated Oct, 2023</font>

### Import the libraries for data analysis and visualization

In [ ]:
import pandas as pd             # Helps with data manipulation
import matplotlib.pyplot as plt # Helps with data visualization
import numpy as np              # Helps with data computation

### Create a list of the CSV file paths for each month of data

In [ ]:
# Create a list of the CSV file paths
csv_file_paths = [
    "202203-divvy-tripdata.csv",
    "202204-divvy-tripdata.csv",
    "202205-divvy-tripdata.csv",
    "202206-divvy-tripdata.csv",
    "202207-divvy-tripdata.csv",
    "202208-divvy-tripdata.csv",
    "202209-divvy-tripdata.csv",
    "202210-divvy-tripdata.csv",
    "202211-divvy-tripdata.csv",
    "202212-divvy-tripdata.csv",
    "202301-divvy-tripdata.csv",
    "202302-divvy-tripdata.csv",
]

### Read the CSV files into Pandas data frames and merge them into one data frame

In [ ]:
# Read the CSV files into Pandas data frames
# It does that so by iterating over the list we created above
df_list = [pd.read_csv(file_path) for file_path in csv_file_paths]

# Merge the data frames using the `pd.concat()` function
merged_df = pd.concat(df_list, ignore_index=True)

# Write the merged data frame to a new CSV file
merged_df.to_csv("merged.csv", index=False)

### Explore the merged data frames and its columns

In [ ]:
merged_df

### Remove all duplicate rows from the merged data frames and save it to a new CSV file

In [ ]:
# Read the merged CSV file
data = pd.read_csv("merged.csv")

# Remove all duplicate rows
merged_deduplicated = data.drop_duplicates()

# Write the merged data frame to a new CSV file
merged_deduplicated.to_csv("merged_deduplicated.csv", index=False)

### Remove the unnecessary columns that contain location data as we will not need it for this analysis

In [ ]:
# Read the merged CSV file
data = pd.read_csv("merged_deduplicated.csv")

# Remove the unnecessary columns
data = data.drop(columns=["start_lat", "start_lng", "end_lat", "end_lng"])

data

### Add new columns for date, month, day, year, day of week, and ride length

In [ ]:
# Add a column for the date
data["date"] = pd.to_datetime(data["started_at"])

# Add a column for the month
data["month"] = pd.to_datetime(data["date"], format='%m').dt.month_name()

# Add a column for the day
data["day"] = data["date"].dt.day

# Add a column for the year
data["year"] = data["date"].dt.year

# Add a column for the day of the week
data["day_of_week"] = data["date"].dt.day_name()

# Add a column for the ride length in seconds
data["ended_at"] = pd.to_datetime(data["ended_at"])
data["started_at"] = pd.to_datetime(data["started_at"])
data["ride_length_sec"] = (data["ended_at"] - data["started_at"]).dt.total_seconds()


# Filter out rows where the start_station_name is "HQ QR" or the ride_length is less than 0
# The data frame includes a few hundred entries when bikes were taken out of docks and checked
# for quality by Divvy or ride_length was negative
data = data[~((data["start_station_name"] == "HQ QR") | (data["ride_length_sec"] < 0))]

# Write the updated data frame to a new CSV file
data.to_csv("dataCleaned.csv", index=False)

data

### Create a pivot table to summarize the ride length statistics by rider type

In [ ]:
# Create a pivot table to summarize the mean, median, max, and min ride length by member casual
ride_length_summary = data.pivot_table(
    index="member_casual",
    values="ride_length_sec",
    aggfunc=["mean", "median", "max", "min"],
)

# Print the pivot table
print(ride_length_summary)

### Run this before you go any further

In [ ]:
# To avoid SettingWithCopyWarning by not having to modifying the original data frame
data = data.copy()

### Create a pie chart to visualize the proportion of casual vs. member riders

In [ ]:
# Group the data by member_casual and count the number of rides for each group
ride_counts = data.groupby("member_casual").size()

# Create a pie chart using matplotlib.pyplot
plt.figure(figsize=(4, 4))
ride_counts.plot(kind="pie", autopct="%1.f%%", labels=None, colors=["lightseagreen", "salmon"])
plt.ylabel('')
plt.legend(["Casual", "Member"], loc="best")
plt.title("Casual vs. Member Riders")
plt.savefig("pie_chart.png")

### Create a bar chart to visualize the number of rides by rider type and day of the week

In [ ]:
# Define the order of days
days_order = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']

# Convert 'day_of_week' to a categorical type
data['day_of_week'] = pd.Categorical(data['day_of_week'], categories=days_order, ordered=True)

# Create a pivot table to summarize the number of rides by rider type and day of the week
ride_counts = data.pivot_table(
    index="day_of_week",
    columns="member_casual",
    values="ride_length_sec",
    aggfunc=np.size,
)

# Create a bar chart to visualize the number of rides by rider type and day of the week
plt.figure(figsize=(10, 6))
ride_counts.plot(kind="bar", stacked=False, color=['lightseagreen', 'salmon'])
plt.xlabel("Day of the Week")
plt.ylabel("Number of Rides")
plt.title("Casual vs. Member Number of Rides on Different Days of the Week")
plt.xticks(rotation=45)
plt.tight_layout()
plt.savefig("ride_counts_week.png")

### Create a bar chart to visualize the number of rides by rider type and rideable type

In [ ]:
# Create a pivot table to summarize the number of rides by rider type and rideable type
ride_counts_rideable_type = data.pivot_table(
    index="rideable_type",
    columns="member_casual",
    values="ride_length_sec",
    aggfunc=np.size,
)

# Create a bar chart to visualize the number of rides by rider type and rideable type
plt.figure(figsize=(10, 6))
ride_counts_rideable_type.plot(kind="bar", stacked=True, color=['lightseagreen', 'salmon'])
plt.xlabel("Rideable Type")
plt.ylabel("Number of Rides")
plt.title("Casual vs. Member Number of Rides on Different Rideable Types")
plt.ticklabel_format(style='plain', axis='y')
plt.xticks(rotation=45)
plt.tight_layout()
plt.savefig("ride_counts_rideable_type.png")

### Create a bar chart to visualize the number of rides by rider type and month

In [ ]:
# Define the order of months
months_order = ['January', 'February', 'March', 'April', 'May', 'June', 'July', 'August', 'September', 'October', 'November', 'December']

# Convert 'month' to a categorical type
data['month'] = pd.Categorical(data['month'], categories=months_order, ordered=True)

# Create a pivot table to summarize the number of rides by rider type and month
ride_counts_month = data.pivot_table(
    index="month",
    columns="member_casual",
    values="ride_length_sec",
    aggfunc=np.size,
)

# Create a bar chart to visualize the number of rides by rider type and month
plt.figure(figsize=(10, 6))
ride_counts_month.plot(kind="bar", stacked=False, color=['lightseagreen', 'salmon'])
plt.xlabel("Month")
plt.ylabel("Number of Rides")
plt.title("Casual vs. Member Number of Rides on Different Months")
plt.xticks(rotation=45)
plt.tight_layout()
plt.savefig("ride_counts_month.png")

### Create a bar chart to visualize the average ride duration by rider type and day of the week

In [ ]:
# Define the order of days
days_order = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']

# Convert 'day_of_week' to a categorical type
data['day_of_week'] = pd.Categorical(data['day_of_week'], categories=days_order, ordered=True)

# Create a pivot table to summarize the average ride duration by rider type and day of the week
ride_counts = data.pivot_table(
    index="day_of_week",
    columns="member_casual",
    values="ride_length_sec",
    aggfunc=np.mean,
)

# Create a bar chart to visualize the average ride duration by rider type and day of the week
plt.figure(figsize=(10, 6))
ride_counts.plot(kind="bar", stacked=False, color=['lightseagreen', 'salmon'])
plt.xlabel("Day of the Week")
plt.ylabel("Average Ride Duration")
plt.title("Casual vs. Member Average Ride Duration on Different Days of the Week")
plt.xticks(rotation=45)
plt.tight_layout()
plt.savefig("ride_average_duration_week.png")